In [ ]:
import pandas as pd
import openpyxl
import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

In [ ]:
pre_cleaning_df = pd.read_excel('otomoto_v2.xlsx')
cols_to_drop = ['Homologacja ciężarowa','Spalanie W Cyklu Mieszanym','Wartość wykupu','Liczba pozostałych rat','Opłata początkowa','Miesięczna rata','lub do (przebieg km)','Okres gwarancji producenta',
                'VAT marża','Gwarancja dealerska (w cenie)','Możliwość finansowania','Numer rejestracyjny pojazdu','Spalanie Poza Miastem','Emisja CO2','Spalanie W Mieście']
pre_cleaning_df.drop(cols_to_drop, axis=1, inplace= True)

In [ ]:
post_cleaning_df = pd.read_excel('otomoto_v2_data_cleaning.xlsx')

In [ ]:
df_inner = pd.merge(pre_cleaning_df, post_cleaning_df, on = 'Link', how = 'inner')

In [ ]:
#returns 0/1 df indicating differences between pre and post cleaning tables
def total_compare(df_to_analyze = df_inner):
    
    cols = pre_cleaning_df.columns.to_list()
    cols.remove('Link')
    df_cols = pd.DataFrame(columns=cols)
 
    df_rows = df_to_analyze['Link']
 
    df_empty = pd.concat([df_rows,df_cols],axis=1)
    df_empty.set_index('Link', inplace=True)
 
    nulls = ['nan','NaT']
 
    for id,row in df_empty.iterrows():
        for col in cols:
            old_col = str(col)+'_x'
            new_col = str(col)+'_y'
 
            old_val = df_inner[df_inner['Link'] == id][old_col].iloc[0]
            new_val = df_inner[df_inner['Link'] == id][new_col].iloc[0]
 
            if old_val != new_val and not(str(old_val) in nulls and str(new_val) in nulls):
                df_empty.loc[id,col] = 1
            else:
                df_empty.loc[id,col] = 0
 
    df_empty.loc['TOTAL_COL'] = df_empty.sum()
    df_empty['TOTAL_ROW'] = df_empty.sum(axis=1)
 
    return df_empty


In [ ]:
result_df = total_compare()

In [ ]:
result_df.to_excel('otomoto_v2_data_cleaning_test.xlsx', index = True)

In [ ]:
#returns df for one ID (link) with columns where there were differences
def both_compare_id(ID):
    cols = pre_cleaning_df.columns.to_list()
    cols.remove('Link')
 
    data = df_inner[df_inner['Link'] == ID]
 
    df = pd.DataFrame(index=['old','new'])
    nulls = ['nan','NaT']
 
    for col in cols:
        old = str(col)+'_x'
        new = str(col)+'_y'
        if data[old].iloc[0] != data[new].iloc[0] and not(str(data[old].iloc[0]) in nulls and str(data[new].iloc[0]) in nulls):
            old_val = df_inner[df_inner['Link'] == ID][old].iloc[0]
            new_val = df_inner[df_inner['Link'] == ID][new].iloc[0]
            val_lst = [old_val,new_val]
            df[col] = val_lst
    return df

In [ ]:
both_compare_id('')